IMPORTS and GLOBALS

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from tqdm.notebook import tqdm
from livelossplot import PlotLosses 
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM
import os

# use Samuel's directory instead
model_path: str = "/home/samuel/research/llmattacks/llm-attacks/DIR/llama-2/llama/Llama-2-7b-chat-hf"
device: str = "cuda:0"
num_steps: int = 2000
num_tokens: int = 100 # Make this smaller; such as 100
step_size: float = 0.1 # BAL even using 0.1 does NOT solve the gradient explosion problem

seed: int = 42
load_dataset = True
# verbose = True
# early_stopping = True

In [ ]:
def load_model_and_tokenizer(model_path, tokenizer_path=None, device="cuda:0", **kwargs):
    # from llm-attacks
    model = (
        AutoModelForCausalLM.from_pretrained(
            model_path, torch_dtype=torch.float16, trust_remote_code=True, **kwargs
        ).to(device).eval()
    )

    tokenizer_path = model_path if tokenizer_path is None else tokenizer_path

    tokenizer = AutoTokenizer.from_pretrained(
        tokenizer_path, trust_remote_code=True, use_fast=False
    )

    if "llama-2" in tokenizer_path:
        tokenizer.pad_token = tokenizer.unk_token
        tokenizer.padding_side = "left"
    if not tokenizer.pad_token:
        tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

def get_embedding_matrix(model):
    # from llm-attacks
    if isinstance(model, LlamaForCausalLM):
        return model.model.embed_tokens.weight
    else:
        raise ValueError(f"Unknown model type: {type(model)}")

In [ ]:
if seed is not None:
        torch.manual_seed(seed)

model, tokenizer = load_model_and_tokenizer(
        model_path, low_cpu_mem_usage=True, use_cache=False, device=device
    )

In [ ]:
def get_tokens(input_string):
    return torch.tensor(tokenizer(input_string)["input_ids"], device=device)


def create_one_hot_and_embeddings(tokens, embed_weights):
    one_hot = torch.zeros(
        tokens.shape[0], embed_weights.shape[0], device=device, dtype=embed_weights.dtype
    )
    one_hot.scatter_(
        1,
        tokens.unsqueeze(1),
        torch.ones(one_hot.shape[0], 1, device=device, dtype=embed_weights.dtype),
    )
    embeddings = (one_hot @ embed_weights).unsqueeze(0).data
    return one_hot, embeddings

In [ ]:
def calc_loss(model, embeddings_user, embeddings_adv, embeddings_target, targets):
    full_embeddings = torch.hstack([embeddings_user, embeddings_adv, embeddings_target])
    logits = model(inputs_embeds=full_embeddings).logits
    loss_slice_start = len(embeddings_user[0]) + len(embeddings_adv[0])
    loss = nn.CrossEntropyLoss()(logits[0, loss_slice_start - 1 : -1, :], targets)
    return loss, logits[:, loss_slice_start-1:, :]
    # return loss, logits.to(torch.float16)

In [ ]:
def get_nonascii_toks(tokenizer):
    def is_ascii(s):
        return s.isascii() and s.isprintable()
    non_ascii_toks = []
    for i in range(3, tokenizer.vocab_size):
        if not is_ascii(tokenizer.decode([i])):
            non_ascii_toks.append(i)
    
    if tokenizer.bos_token_id is not None:
        non_ascii_toks.append(tokenizer.bos_token_id)
    if tokenizer.eos_token_id is not None:
        non_ascii_toks.append(tokenizer.eos_token_id)
    if tokenizer.pad_token_id is not None:
        non_ascii_toks.append(tokenizer.pad_token_id)
    if tokenizer.unk_token_id is not None:
        non_ascii_toks.append(tokenizer.unk_token_id)
    
    return torch.tensor(non_ascii_toks).to(device)

# # Test method
# non_ascii_toks = get_nonascii_toks(tokenizer)
# print(non_ascii_toks.tolist())


def get_only_ASCII_one_hot_adv():

    non_ascii_toks = get_nonascii_toks(tokenizer).tolist()
    no_of_ascii_toks = tokenizer.vocab_size - len(non_ascii_toks)
    # print(len(non_ascii_toks))
    # print(tokenizer.vocab_size)
    # print(no_of_ascii_toks)
    # for tok_id in non_ascii_toks:
    #     print(tokenizer.decode(tok_id), end=' ')
    # Assuming device is already defined
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Step 1: Create the 20x32000 tensor randomly initialized and Initialize weights at the center of the simplex
    one_hot_adv = torch.rand(20, 32000, dtype=torch.float16).to(device=device)
    # dims = one_hot_adv.size(-1)
    one_hot_adv.data.fill_(1 / no_of_ascii_toks)
    # Step 2: Assuming token_ids_init is a tensor of shape (20, k) where k is the number of indices per row
    # top_token_ids_tensor = select_topk_adv_token_ids(input_files, input_directory)
    # print('size:', top_token_ids_tensor.size())
    # print('token_ids:', top_token_ids_tensor)
    # Repeat the tensor 20 times along a new dimension and then reshape
    non_ascii_toks_tensor = torch.tensor(non_ascii_toks).to(device=device)
    top_token_ids_tensor_2d = non_ascii_toks_tensor.unsqueeze(0).repeat(20, 1)
    
    # Step 3: Create a mask with the same shape as one_hot_adv, initialized to zero
    mask = torch.ones_like(one_hot_adv, dtype=torch.float16)

    # Step 4: Use token_ids_init to set the corresponding indices in the mask to 1
    # We use gather and scatter operations for this
    mask.scatter_(1, top_token_ids_tensor_2d, 0.0)

    # Step 5: Apply the mask to one_hot_adv
    one_hot_adv = one_hot_adv * mask
    
    return one_hot_adv

# # Example Usage
# one_hot_adv = select_only_ASCII_token_ids()
# print(one_hot_adv)

# sum = torch.sum(one_hot_adv, dim=1)
# max = torch.max(one_hot_adv, dim=1)
# min = torch.min(one_hot_adv, dim=1)
# print(sum,',', max, ',', min)

def count_common_words(str1, str2):
    # Split the strings into words
    words1 = set(str1.split())
    words2 = set(str2.split())

    # Find the intersection of the two sets to get the common words
    common_words = words1.intersection(words2)

    # Return the number of common words
    return len(common_words)

In [ ]:
# def get_clipped_embedding(embed_weights, generated_embeddings):
#     # Step 1: Compute the mean embedding across all samples (dim=0 to get mean for each dimension)
#     mean_embedding = torch.mean(embed_weights, dim=0)
#     # Step 2: Compute the variance for each dimension
#     variance_embedding = torch.var(embed_weights, dim=0, unbiased=False)
#     # Compute the standard deviation for each dimension
#     std_devs = torch.sqrt(variance_embedding)
#     # Set clipping thresholds based on the mean and standard deviations
#     lower_thresholds = mean_embedding - 3 * std_devs
#     upper_thresholds = mean_embedding + 3 * std_devs
#     # Example generated embeddings (replace with your actual data)
#     # Clip the generated embeddings based on the thresholds for each dimension
#     clipped_embeddings = torch.max(torch.min(generated_embeddings, upper_thresholds), lower_thresholds)
#     return clipped_embeddings

BEGIN ATTACK HERE

In [ ]:
import csv
embed_weights = get_embedding_matrix(model)
dataset_path: str = "/home/samuel/research/llmattacks/llm-attacks/data/advbench/harmful_behaviors.csv"
if load_dataset:
    reader = csv.reader(open(dataset_path, 'r'))
    next(reader)
else:
    print("Error!")
harmful_behaviors = list(reader)[0:1]
# We can hard-code this safe_target string; probably
safe_target: str = "I cannot fulfill your request. I'm just an AI."
print(len(harmful_behaviors))
for each_behavior in harmful_behaviors:
    print(each_behavior)
    # user_prompt, target = each_behavior
    # print('Query:', user_prompt)
    # user_prompt_tokens = get_tokens(user_prompt)    
    # _, embeddings_user = create_one_hot_and_embeddings(user_prompt_tokens, embed_weights)
    # generated_output =  model.generate(inputs_embeds=embeddings_user, max_length=len(user_prompt_tokens)).squeeze()
    # generated_output_string = tokenizer.decode(generated_output, skip_special_tokens=True)
    # print('Response:', generated_output_string,end='\n\n\n')
    # each_behavior.append(generated_output_string)

In [ ]:
# Optimize for one Target(Unsafe->Safe); but plot both the losses together.
# Update the one_hot encodings using Gradient Descent.
# Use random initialization for Adversarial One Hot
from math import *
import json 
import math
import pandas as pd
import torch.nn.functional as F
embed_weights = get_embedding_matrix(model)
# reader = [[user_prompt, target]]
optimization_results = []
# Set your regularization parameter
initial_coefficient = 1e-4
final_coefficient = 1e-2
# Create the following directories if they do NOT exist
directories = ["./CSV_Files", "./JSON_Files", "./TORCH_Files"]
for directory in directories:
    # Create the directory if it doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory '{directory}' created.")
    else:
        print(f"Directory '{directory}' already exists.")
# JSON file to dump the model generated outputs 
json_file_path = "./JSON_Files/output_EGD_w_Entropy(only_ASCII_init)_"+str(len(harmful_behaviors))+"_behaviors("+str(num_steps)+"Iterations).json"

for row in tqdm(harmful_behaviors, desc="Optimizing prompts"):
    iteration_result = {}
    user_prompt, unsafe_target = row
    print(user_prompt)
    target = unsafe_target
    user_prompt_tokens = get_tokens(user_prompt)    
    target_tokens = get_tokens(target)[1:]
    plotlosses = PlotLosses(groups={'loss': ['unsafe_loss', 'safe_loss']})
    # Create both embeddings and one_hot for Safe and Unsafe target Seperate (To plot the losses)
    unsafe_target_tokens = get_tokens(unsafe_target)[1:]    
    safe_target_tokens = get_tokens(safe_target)[1:]
    # adv_suffix_tokens = get_tokens(adv_suffix)[1:]
    one_hot_unsafe_target, embeddings_unsafe_target = create_one_hot_and_embeddings(unsafe_target_tokens, embed_weights)
    one_hot_safe_target, embeddings_safe_target = create_one_hot_and_embeddings(safe_target_tokens, embed_weights)

    # Initialize one_hot encodings and embedding vectors for the user prompts and targets 
    one_hot_inputs, embeddings_user = create_one_hot_and_embeddings(user_prompt_tokens, embed_weights)
    one_hot_target, embeddings_target = create_one_hot_and_embeddings(target_tokens, embed_weights)
    best_disc_loss = np.inf
    best_loss_at_epoch = 0
    # Initialize the adversarial one_hot encodings using ONLY ASCII tokens
    one_hot_adv = get_only_ASCII_one_hot_adv()
    # Alternatively, initialize adversarial one_hot encodings using adv_suffix_init; This gives a Constant Loss value.
    # one_hot_adv, embeddings_adv = create_one_hot_and_embeddings(adv_suffix_tokens, embed_weights) 
    effective_adv_one_hot = one_hot_adv.detach()
    effective_adv_embedding = (one_hot_adv @ embed_weights).unsqueeze(0)
    one_hot_adv.requires_grad_()
    # Initialize Adam optimizer with user-defined epsilon value
    optimizer = optim.Adam([one_hot_adv], lr=step_size, eps=1e-4) 
    scheduler_cycle = 0
    # Specify the filename
    last_three_words = lambda text: '_'.join(text.split()[-3:])
    first_three_words = lambda text: '_'.join(text.split()[0:3])
    csv_filename = './CSV_Files/output_EGD_w_Entropy(only_ASCII_init)_'+last_three_words(user_prompt)+'('+str(num_steps)+'_iterations).csv'        
    # Generate column names
    column_names = ['epoch', 'cycle', 'learning_rate', "entropy_term", "unsafe_loss", "safe_loss"] 
                    # 'continuous_loss','discrete_loss'] 
    #                'l1_loss', 'total_loss', 
    # Adding 'max_1' to 'max_20' column names using a loop
    for i in range(1, 21):
        column_names.append(f'max_{i}')
    for i in range(1, 21):
        column_names.append(f'token_id_{i}')
    # Create an empty DataFrame
    df = pd.DataFrame(columns=column_names)

    PHASE_2: bool=False
    BREAK_IT: bool=False

    for epoch_no in tqdm(range(num_steps)):
        optimizer.zero_grad()

        embeddings_adv = (one_hot_adv @ embed_weights).unsqueeze(0)
        ce_loss, logits = calc_loss(model, embeddings_user, embeddings_adv, embeddings_target, one_hot_target)
        continuous_loss = ce_loss.detach().cpu().item()
        # Break the loop if loss becomes "NaN"
        if math.isnan(continuous_loss):
            print("The value is NaN.")
            BREAK_IT=True # Go to the Next Behavior
        # Use the generate() fuction to check whether attack is successful.
        tokens_pred = logits.argmax(2)
        word_count = len(set(target.split()))
        output_str = tokenizer.decode(tokens_pred[0][:].cpu().numpy())
        common_word_count = count_common_words(output_str, target)
        sucess = common_word_count>=(word_count-1) and continuous_loss<0.2
        # OR: sucess = common_word_count>=word_count or continuous_loss<0.2
        print("common_word_count:",common_word_count,end=" for ")
        print("output_str:", output_str)
        if sucess:        
            # clipped_embeddings_adv = get_clipped_embedding(embed_weights, embeddings_adv)
            final_prompt_embeds = torch.hstack([embeddings_user, embeddings_adv])
            # final_prompt_embeds = torch.hstack([embeddings_user, embeddings_adv])
            outputs = []
            generation_loop = range(5)
            for loop in tqdm(generation_loop, desc="Generating outputs", leave=False):
                generated_output =  model.generate(inputs_embeds=final_prompt_embeds, max_length=num_tokens).squeeze()
                # generated_output =  model.generate(inputs_embeds=final_prompt_embeds, max_length=num_tokens, do_sample=False).squeeze()
                generated_output_string = tokenizer.decode(generated_output, skip_special_tokens=True)
                outputs.append(generated_output_string)

            iteration_result = {
                "harmful-behaviour": user_prompt,
                "target": target,
                "epoch_no": epoch_no,
                "continuous_loss": continuous_loss,
                "outputs": outputs
            }
            optimization_results.append(iteration_result)
            # Update the JSON File
            with open(json_file_path, "w") as f:
                json.dump(optimization_results, f, indent=4)
            # Save the Tensor/vector you have at this point
            # Go to the next phase
            if PHASE_2==False:
                # Save the Tensor/vector you have at this point
                torch_file_name = './TORCH_Files/EGD_w_Entropy_Unsafe_1_hot_adv_'+last_three_words(user_prompt)+'_'+first_three_words(unsafe_target)+'.pt'                
                torch.save(one_hot_adv.data, torch_file_name)
                target = safe_target
                target_tokens = get_tokens(target)[1:]
                one_hot_target, embeddings_target = create_one_hot_and_embeddings(target_tokens, embed_weights)
                PHASE_2=True
            else : # PHASE_2 is True
                torch_file_name = './TORCH_Files/EGD_w_Entropy_Safe_1_hot_adv_'+last_three_words(user_prompt)+'_'+first_three_words(safe_target)+'.pt'                
                torch.save(one_hot_adv.data, torch_file_name)
                BREAK_IT=True
        # Use the generate() fuction to check whether attack is successful.
        # ce_loss.backward()
        ################# Usual entropy #################
        # Calculate H(X) = − ∑_{i=1}^{L} ∑_{j=1}^{|T|} X_{ij} (log X_{ij} −1)
        # Adding a small epsilon to avoid log(0) which is undefined
        eps = 1e-12
        log_one_hot = torch.log(one_hot_adv.to(dtype=torch.float32) + eps)
        # Compute the modified entropy
        entropy_per_row = -one_hot_adv.to(dtype=torch.float32) * (log_one_hot - 1)
        entropy_term = entropy_per_row.sum()
        # Calculate the annealed coefficient, 𝜖
        # Use Exponential Scheduling
        reg_coefficient = initial_coefficient * (final_coefficient / initial_coefficient) ** (epoch_no / (num_steps - 1))   
        entropy_term *= reg_coefficient 
 
        # Regularized loss: F(X) - epsilon * H(X)
        regularized_loss = ce_loss - entropy_term
        # Backpropagate the regularized loss
        regularized_loss.backward()

        # Clip the gradients(before update)
        # torch.nn.utils.clip_grad_norm_([one_hot_adv], max_norm=1.0)

        # Copy the gradients
        grads = one_hot_adv.grad.clone()        
        # Do NOT Normalize gradients
        # grads = grads / grads.norm(dim=-1, keepdim=True)

        # Exponentiated Gradient Descent update
        with torch.no_grad():
            # one_hot_adv.data *= torch.exp(-eta * grads)
            one_hot_adv.data *= torch.exp(-step_size * grads)
            one_hot_adv.data /= one_hot_adv.data.sum(dim=1, keepdim=True)

        ## Convex Check
        sum = torch.sum(one_hot_adv, dim=1)
        max = torch.max(one_hot_adv, dim=1)
        non_zero_count = torch.count_nonzero(one_hot_adv, dim=1)

        # Clip the gradients(after update)
        torch.nn.utils.clip_grad_norm_([one_hot_adv], max_norm=1.0)
        # Rest the gradients
        model.zero_grad()
        one_hot_adv.grad.zero_()
        
        # # Discretization part
        max_values = torch.max(one_hot_adv, dim=1)
        adv_token_ids = max_values.indices
        # one_hot_discrete = torch.zeros(
        #     adv_token_ids.shape[0], embed_weights.shape[0], device=device, dtype=embed_weights.dtype
        # )
        # one_hot_discrete.scatter_(
        #     1,
        #     adv_token_ids.unsqueeze(1),
        #     torch.ones(one_hot_adv.shape[0], 1, device=device, dtype=embed_weights.dtype),
        # )
        # Update plotlosses with both Unsafe and Safe loss
        ce_loss, _ = calc_loss(model, embeddings_user, embeddings_adv, embeddings_unsafe_target, one_hot_unsafe_target)
        unsafe_loss = ce_loss.detach().cpu().item()
        ce_loss, _ = calc_loss(model, embeddings_user, embeddings_adv, embeddings_safe_target, one_hot_safe_target)
        safe_loss = ce_loss.detach().cpu().item()
        plotlosses.update({
            "unsafe_loss": unsafe_loss,
            "safe_loss": safe_loss
        })
        plotlosses.send()

        # # Dump Ouput values to a CSV file
        # # Convert max_values to a NumPy array
        max_values_array = max_values.values.detach().cpu().numpy()
        token_ids_array = adv_token_ids.detach().cpu().numpy()
        # Compute Cycle using the scheduler's state
        # if scheduler_state['T_cur'] == 0:
        #         scheduler_cycle += 1
        # Create the Initial array       
        prepend_array = np.array([epoch_no, scheduler_cycle, step_size,
                                  entropy_term.detach().cpu().item(), 
                                  unsafe_loss, safe_loss]) 
        
        # Concatenate the arrays
        row = np.concatenate((prepend_array, max_values_array, token_ids_array))
        new_row = pd.Series(row, index=df.columns)
        df = pd.concat([df, new_row.to_frame().T], ignore_index=True)
        # Save log data to CSV file periodically
        if epoch_no % 10 == 0:
            df.to_csv(csv_filename, index=False)
        # Something went wrong; so break it and go to the next behavior.
        if BREAK_IT==True:
            break
    # End of optimizations        
    # Write to CSV file
    df.to_csv(csv_filename, index=False)


In [ ]:
# File paths to the .pt files

for row in harmful_behaviors:
    iteration_result = {}
    user_prompt, _ = row
    print(f'\n\n{user_prompt}')
    print('________________________________________________________________________________')
    tensor_file_unsafe = './TORCH_Files/EGD_w_Entropy_Unsafe_1_hot_adv_'+last_three_words(user_prompt)+'_'+first_three_words(unsafe_target)+'.pt'
    tensor_file_safe = './TORCH_Files/EGD_w_Entropy_Safe_1_hot_adv_'+last_three_words(user_prompt)+'_'+first_three_words(safe_target)+'.pt'
    tensor_filenames = [tensor_file_unsafe, tensor_file_safe]
    loaded_tensors = []
    skip: bool=False
    for filename in tensor_filenames:
        if os.path.exists(filename):
            loaded_tensors.append(torch.load(filename))
        else:
            print(f"File {filename} does not exist. Skipping...")
            skip=True
    if skip==True: # GO to the NExt Iteration
        continue
    # Load the tensors from the .pt files
    # loaded_tensors = [torch.load(filename) for filename in tensor_filenames]

    # Now, loaded_tensors is a list containing the loaded tensors
    tensor1 = loaded_tensors[0]
    tensor2 = loaded_tensors[1]

    # Move tensors to CPU and convert to float32 if necessary
    tensor1_cpu = tensor1.cpu().float()
    tensor2_cpu = tensor2.cpu().float()

    # Example usage: Compute cosine similarity and L2 distance
    import torch.nn.functional as F

    # Compute cosine similarity
    cosine_similarity = F.cosine_similarity(tensor1_cpu, tensor2_cpu, dim=1)

    # Compute L2 distance using torch.cdist
    l2_distance = torch.cdist(tensor1_cpu.unsqueeze(0), tensor2_cpu.unsqueeze(0), p=2).squeeze(0)

    # Print results
    print("Cosine Similarity:")
    print(cosine_similarity)
    print("\nL2-norm Distance size")
    print(l2_distance.shape)

    # If you only need the distance between corresponding rows
    diagonal_l2_distance = l2_distance.diag()
    print("L2-norm Distance between corresponding rows:")
    print(diagonal_l2_distance)

    # Compute L1 distance
    # Compute the L1-norm distance
    l1_distance = torch.cdist(tensor1_cpu.unsqueeze(0), tensor2_cpu.unsqueeze(0), p=1).squeeze(0)
    # torch.abs(tensor1 - tensor2).sum()
    print("\nL1-norm distance size:\n", l1_distance.shape)
    print("L1-norm Distance between corresponding rows:")
    print(l1_distance.diag())

## Additional Codes

In [ ]:
# # Save the tensor as a CSV file
# def save_tensor_to_csv(tensor, filename):
#     # Convert the tensor to a NumPy array
#     numpy_array = tensor.cpu().numpy()
#     # Convert the NumPy array to a DataFrame
#     df = pd.DataFrame(numpy_array)
#     # Save the DataFrame to a CSV file
#     df.to_csv(filename, index=False)



# csv_file_unsafe = './outputs/Unsafe_1_hot_adv_'+first_three_words(unsafe_target)+'.csv'
# csv_file_safe = './outputs/Safe_1_hot_adv_'+first_three_words(safe_target)+'.csv'
# save_tensor_to_csv(tensor1_cpu, csv_file_unsafe)
# save_tensor_to_csv(tensor2_cpu, csv_file_safe)

In [ ]:
# # Compute cosine_similarity and l2-distance

# # Move tensors to CPU if necessary
# tensor1_cpu = one_hot_adv_list[0].cpu()
# tensor2_cpu = one_hot_adv_list[1].cpu()

# # Compute cosine similarity
# cosine_similarity = F.cosine_similarity(tensor1_cpu, tensor2_cpu, dim=1)

# # Compute L2 distance using torch.cdist
# # Move tensors to CPU and convert to float32 if necessary
# tensor1_cpu = tensor1_cpu.float()
# tensor2_cpu = tensor2_cpu.float()
# l2_distance = torch.cdist(tensor1_cpu, tensor2_cpu, p=2)
# diagonal_l2_distance = l2_distance.diag()
# # Print results
# print("Cosine Similarity:")
# print(cosine_similarity)
# print("\nL2-norm Distance:")
# print(diagonal_l2_distance)

In [ ]:
# # After it's done, compute the similarity and distance between the one_hot_adv_Safe and one_hot_adv_Unsafe
# print(len(one_hot_adv_list))
# # Use the Last embeddings_adv
# for one_hot_adv in one_hot_adv_list:
#     embeddings_adv = (one_hot_adv @ embed_weights).unsqueeze(0)
#     final_prompt_embeds = torch.hstack([embeddings_user, embeddings_adv])
#     outputs = []
#     generation_loop = range(10)
#     for i in tqdm(generation_loop, desc="Generating outputs", leave=False):
#         generated_output =  model.generate(inputs_embeds=final_prompt_embeds, max_length=num_tokens).squeeze()
#         generated_output_string = tokenizer.decode(generated_output, skip_special_tokens=True)
#         print(f"#{i}\n",generated_output_string)
#         outputs.append(generated_output_string)